In [ ]:
import numpy as np
import pandas as pd
from keras.applications.resnet50 import ResNet50, decode_predictions
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model
from keras.preprocessing.image import image
from tqdm import tqdm
import h5py
from helper_functions import get_filenames, get_batches
from generators import pred_generator_model4
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.base import clone
from IPython.display import clear_output
import config

In [ ]:
#Dirs with photos
TRAIN_DIR = config.TRAIN_DIR
#TEST_DIR = "D:/test_photos/"

#CSV files with photos ids and corresponding business ids 
#test_photo_to_biz = pd.read_csv("D:/test_photo_to_biz.csv")
train_photo_to_biz = config.train_photo_to_biz

#CSV file with labels corresponding to business
labels = config.labels
lbs_pd = config.labels

In [ ]:
batches, lbs = get_batches(train_photo_to_biz, lbs_pd)

In [ ]:
#Using pretrained model
ResNet_model = ResNet50(weights='imagenet', include_top = True)

filenames = get_filenames(TRAIN_DIR)
businesses = []
gen = pred_generator_model4(filenames, businesses, train_photo_to_biz)

In [ ]:
preds = []
for i in tqdm(range(117421)):
    preds.append(ResNet_model.predict(next(gen), batch_size=2))

In [ ]:
file = h5py.File("model4_output.h5", "w")
file.create_dataset("labels", data=np.array(preds))
file.create_dataset("businesses", data=np.array(businesses))
file.close()

In [ ]:
file = h5py.File("model4_output.h5", "r")
businesses = file["businesses"][:]
imagenet_labels = file["labels"][:]
file.close()

In [ ]:
lbs = imagenet_labels.reshape(234842, 1000)
decoded = decode_predictions(lbs)
statistic_dict = {}

In [ ]:
for i in range(len(decoded)):
    for j in range(len(decoded[i])):
        if decoded[i][j][1] in statistic_dict:
            statistic_dict[decoded[i][j][1]] += np.array([1.,decoded[i][j][2]])
        else:
            statistic_dict[decoded[i][j][1]] = np.array([1.,decoded[i][j][2]])

In [ ]:
sorted(statistic_dict.items(), key = lambda kv:(kv[1][0], kv[0]), reverse=True)

In [ ]:
sorted(statistic_dict.items(), key = lambda kv:(kv[1][1], kv[0]), reverse=True)

In [ ]:
X = []
Y = []
n = 0
for i in np.unique(businesses):
    if type(list(labels[labels["business_id"]==i]["labels"])[0])!=float:
        Y.append(list(map(lambda x:int(x), list(labels[labels["business_id"]==i]["labels"])[0].split())))
        X.append(np.sum(lbs[np.where(businesses==i)], axis=0))
        clear_output()
        n+=1
        print(n)